# AI easy challenge writeup

## Importing the libraries

In [2]:
# import libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
import numpy as np

## Model exploration

Before trying to invert the model and reconstruct the flag image, we need to explore the model to understand its properties.

In [3]:
model = tf.keras.models.load_model("../train/model.keras")

We print the summary to understand the overall structure

In [3]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 300, 300, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 299, 299, 8)    │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 149, 149, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 177608)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       710,436 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            10 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,131,460 (8.13 MB)

 Trainable params: 710,486 (2.71 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,420,974 (5.42 MB)

We then print also the model configuration, from which we can understand that for the output layer the softmax function was used. 

In [4]:
model.get_config()

{'name': 'sequential',
 'trainable': True,
 'dtype': {'module': 'keras',
  'class_name': 'DTypePolicy',
  'config': {'name': 'float32'},
  'registered_name': None},
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_shape': (None, 300, 300, 1),
    'dtype': 'float32',
    'sparse': False,
    'name': 'input_layer'},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Rescaling',
   'config': {'name': 'rescaling',
    'trainable': True,
    'dtype': {'module': 'keras',
     'class_name': 'DTypePolicy',
     'config': {'name': 'float32'},
     'registered_name': None},
    'scale': 0.00392156862745098,
    'offset': 0.0},
   'registered_name': None,
   'build_config': {'input_shape': [None, 300, 300, 1]}},
  {'module': 'keras.layers',
   'class_name': 'Conv2D',
   'config': {'name': 'conv2d',
    'trainable': True,
    'dtype': {'module': 'keras',
     'class_name': 'DTypePolicy',
     'config': {'name': 'float32'},
     're

Lastly, we print also the optimizer configuration, that leads us to the Adam optimizer.

In [5]:
with open("../train/optimizer_config.json", "r") as f:
    print(f.read())

{"name": "adam", "learning_rate": 0.0010000000474974513, "weight_decay": null, "clipnorm": null, "global_clipnorm": null, "clipvalue": null, "use_ema": false, "ema_momentum": 0.99, "ema_overwrite_frequency": null, "loss_scale_factor": null, "gradient_accumulation_steps": null, "beta_1": 0.9, "beta_2": 0.999, "epsilon": 1e-07, "amsgrad": false}


## Model inversion

The overall goal is to reconstruct the flag image, so to basically invert the model. 
We start by defining the number of epochs (trial and error here can bring us to the correct value or we could apply a threshold on the loss) and the learning rate that we would like to apply at each epoch

In [6]:
# define the number of epochs and the learning rate
epochs = 10

learning_rate = 0.001

Furthermore, to speed up convergence we can standardize the image after each epoch.

In [7]:
def rescale(img):
    return tf.keras.layers.Rescaling(scale=1./255)(img)

From the information that we gathered above, we know the optimizer and the loss function that were applied during the training

In [8]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

The inversion process that we will apply will be the following: 

- Start with a randomly generated image
- For each epoch
    - Use the model to get the prediction of the generated image
    - Calculate the loss with respect to the target class, i.e. a vector of the following form: [1, 0, 0]
    - Calculate the gradient that we need to apply to the image to reach an image that will lead us to the correct classification
    - Apply a part of the gradient (as we do in a normal training step) to the image
    - Standardize the image

We start by defining the workflow of each step in a specific function

In [9]:
def model_inversion_step(img, label):
    # Create a variable to hold the image
    image = tf.Variable(rescale(img))
    # Create the optimizer based on the specified learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    with tf.GradientTape() as tape:
        tape.watch(image)
        # Get the prediction of the model
        pred = model(image, training=False)
        # Calculate the loss
        loss = loss_object(label, pred)
        print(f"Loss: {loss}")
        # Get the gradients of the loss w.r.t to the input image.
        gradient = tape.gradient(loss, image)
        # Update the image with the gradients.
        optimizer.apply_gradients(zip([gradient], [image]))
                
    return image.numpy()

Generate a random image

In [10]:
reconstructed_image = np.random.random((1, 300, 300, 1))
reconstructed_image = tf.convert_to_tensor(reconstructed_image)

Set up the target label, i.e. the vector [1, 0, 0]

In [11]:
target_label = 0
target_label = np.zeros(2)
target_label[0] = 1

Apply our inversion process for the chosen amount of epochs

In [12]:
for i in range(0, epochs):
    print("=======================================")
    print(f"Epoch {i + 1}")
    reconstructed_image = model_inversion_step(reconstructed_image, np.array([target_label]))

Epoch 1
Loss: 0.6556369066238403
Epoch 2
Loss: 0.6555203795433044
Epoch 3
Loss: 0.6555202603340149
Epoch 4
Loss: 0.6555202007293701
Epoch 5
Loss: 0.6555202603340149
Epoch 6
Loss: 0.6555202007293701
Epoch 7
Loss: 0.6555202603340149
Epoch 8
Loss: 0.6555202007293701
Epoch 9
Loss: 0.6555202603340149
Epoch 10
Loss: 0.6555201411247253


Print the final loss and class prediction

In [13]:
pred = model(reconstructed_image, training=False)
predicted_class = np.argmax(pred)
loss = loss_object(np.array([target_label]), pred)
print(f"Final loss: {loss}")

Final loss: 0.6554770469665527


Show the final image, which corresponds to the flag

In [14]:
image = tf.keras.preprocessing.image.array_to_img(reconstructed_image[0])
image.show()
image.save("flag.png")